In [1]:
from sklearn import datasets
import pandas as pd
import numpy as np

In [2]:
iris = datasets.load_iris()

In [3]:
df = pd.DataFrame(iris.data)
df.columns = ["sl", "sw", 'pl', 'pw']

In [4]:
def label(val, *boundaries):
    if (val < boundaries[0]):
        return 'a'
    elif (val < boundaries[1]):
        return 'b'
    elif (val < boundaries[2]):
        return 'c'
    else:
        return 'd'


def toLabel(df, old_feature_name):
    second = df[old_feature_name].mean()
    minimum = df[old_feature_name].min()
    first = (minimum + second)/2
    maximum = df[old_feature_name].max()
    third = (maximum + second)/2
    return df[old_feature_name].apply(label, args= (first, second, third))

In [5]:
df['sl_labeled'] = toLabel(df, 'sl')
df['sw_labeled'] = toLabel(df, 'sw')
df['pl_labeled'] = toLabel(df, 'pl')
df['pw_labeled'] = toLabel(df, 'pw')
df

,sl,sw,pl,pw,sl_labeled,sw_labeled,pl_labeled,pw_labeled
0,5.1,3.5,1.4,0.2,b,c,a,a
1,4.9,3.0,1.4,0.2,a,b,a,a
2,4.7,3.2,1.3,0.2,a,c,a,a
3,4.6,3.1,1.5,0.2,a,c,a,a
4,5.0,3.6,1.4,0.2,a,c,a,a
...,...,...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,c,b,c,d
146,6.3,2.5,5.0,1.9,c,a,c,d
147,6.5,3.0,5.2,2.0,c,b,c,d
148,6.2,3.4,5.4,2.3,c,c,d,d


In [6]:
df.drop(['sl', 'sw', 'pl', 'pw'], axis = 1, inplace = True)

In [7]:
df

,sl_labeled,sw_labeled,pl_labeled,pw_labeled
0,b,c,a,a
1,a,b,a,a
2,a,c,a,a
3,a,c,a,a
4,a,c,a,a
...,...,...,...,...
145,c,b,c,d
146,c,a,c,d
147,c,b,c,d
148,c,c,d,d


In [8]:
set(df['sl_labeled'])

{'a', 'b', 'c', 'd'}

In [9]:
import math

In [10]:
class TreeNode:
    def __init__(self, data,output):
       
        self.data = data
        self.children = {} 
        self.output = output
        
        self.index = -1
        
    def add_child(self,feature_value,object):
        self.children[feature_value] = object

In [11]:
class build_tree:
    def __init__(self):
        
        self.__root = None

    def __frequency_counter(self,Y):
        
        d = {}
        for i in Y:
            if i in d:
                d[i]+=1
            else:
                d[i]=1
        return d


    def __entropy(self,Y):
        
        freq_map = self.__frequency_counter(Y)
        entropy_ = 0
        total = len(Y)
        for i in freq_map:
            p = freq_map[i]/total
            entropy_ += (-p)*math.log2(p)
            
        return entropy_

    def __gain_ratio(self,X,Y,selected_feature):
        # returns the gain ratio
        original = self.__entropy(Y) 
        entropy_after_splitting = 0 
        split_info = 0
        values = set(X[:,selected_feature])
        df = pd.DataFrame(X)
        
        df[df.shape[1]] = Y
        starting_size = df.shape[0] 
        for i in values:
            df1 = df[df[selected_feature] == i]
            current_size = df1.shape[0]
            entropy_after_splitting += (current_size/starting_size)*self.__entropy(df1[df1.shape[1]-1])
            split_info += (-current_size/starting_size)*math.log2(current_size/starting_size)

        
        if split_info == 0 :
            return math.inf 
        

        info_gain = original - entropy_after_splitting
        gain_ratio = info_gain / split_info
        return gain_ratio 



    def __decision_tree(self,X,Y,features,level,classes, all_features=np.array([i for i in df.columns])): 





        
        
        if len(features) == 0:
            print("Level",level)
            freqs = self.__frequency_counter(Y)
            output = None
            max_count = -99999999999999
            for i in classes:
                if i not in freqs:
                    print("Count of",i,"=",0)
                else :
                    if freqs[i] > max_count :
                        output = i
                        max_count = freqs[i]
                    print("Count of",i,"=",freqs[i])

            print("Current Entropy  is =",self.__entropy(Y))          

            print("Reached leaf Node")
            print()
            return TreeNode(None,output)
        
        
        
        if len(set(Y)) == 1:
            print("Level",level)
            output = None
            for class_ in classes:
                if class_ in Y:
                    output = class_
                    print("Count of",class_,"=",len(Y))
                else :
                    print("Count of",class_,"=",0)
            print("Current Entropy is =  0.0")
            print("Reached leaf Node")
            print()
            return TreeNode(None,output)

        
        max_gain = -math.inf
        final_feature = None
        for f in features :
            current_gain = self.__gain_ratio(X,Y,f)

            if current_gain > max_gain:
                max_gain = current_gain
                final_feature = f

        print("Level",level)
        freqs = self.__frequency_counter(Y)
        output = None
        max_count = -math.inf

        for class_ in classes:
            if class_ not in freqs:
                print("Count of",class_,"=",0)
            else :
                if freqs[class_] > max_count :
                    output = class_
                    max_count = freqs[class_]
                print("Count of",class_,"=",freqs[class_])
     
        print("Current Entropy is =",self.__entropy(Y))
        print("Splitting on feature" , all_features[final_feature] , "with gain ratio" , max_gain)
        print()
        

            
        unique_values = set(X[:,final_feature]) 
        df = pd.DataFrame(X)
        
        df[df.shape[1]] = Y

        current_node = TreeNode(final_feature,output)

        
        index  = features.index(final_feature)
        features.remove(final_feature)
        for i in unique_values:
            
            df_new = df[df[final_feature] == i]
            
            node = self.__decision_tree(df_new.iloc[:,0:df_new.shape[1]-1].values,df_new.iloc[:,df_new.shape[1]-1].values,features,level+1,classes)
            current_node.add_child(i,node)

             
        features.insert(index,final_feature)

        return current_node
    
    def fit(self,X,Y):
        
        features = [i for i in range(len(X[0]))]
        classes = set(Y)
        level = 0
        self.__root = self.__decision_tree(X,Y,features,level,classes)
        
    def __predict_for_single_point(self,data,node):
        
        if len(node.children) == 0 :
            return node.output

        val = data[node.data]       
        if val not in node.children :
            return node.output
        
        
        return self.__predict_for_single_point(data,node.children[val])

    def predict(self,X):
        
        Y = np.array([0 for i in range(len(X))])
        for i in range(len(X)):
            Y[i] = self.__predict_for_single_point(X[i],self.__root)
        return Y
    
    def score(self,X,Y):
        
        Y_predicted = self.predict(X)
        counter = 0
        for i in range(len(Y_predicted)):
            if Y_predicted[i] == Y[i]:
                counter+=1
        return counter/len(Y_predicted)

In [12]:
from sklearn.model_selection import train_test_split
x=df.values
y = iris.target
x_train, x_test, y_train, y_test=train_test_split(x, y)
unused_features = set(df.columns)
clf=build_tree()
clf.fit(x_train, y_train)
print()
print("Score=", clf.score(x_test, y_test))

Level 0
Count of 0 = 35
Count of 1 = 44
Count of 2 = 33
Current Entropy is = 1.5733825760581053
Splitting on feature pw_labeled with gain ratio 0.7084071685476648

Level 1
Count of 0 = 0
Count of 1 = 34
Count of 2 = 8
Current Entropy is = 0.7024665512903903
Splitting on feature pl_labeled with gain ratio 0.5865820730734441

Level 2
Count of 0 = 0
Count of 1 = 33
Count of 2 = 2
Current Entropy is = 0.31599713297842463
Splitting on feature sl_labeled with gain ratio 0.11092815024614613

Level 3
Count of 0 = 0
Count of 1 = 19
Count of 2 = 1
Current Entropy is = 0.28639695711595625
Splitting on feature sw_labeled with gain ratio 0.028119477329726954

Level 4
Count of 0 = 0
Count of 1 = 6
Count of 2 = 0
Current Entropy  is = 0.0
Reached leaf Node

Level 4
Count of 0 = 0
Count of 1 = 12
Count of 2 = 1
Current Entropy  is = 0.39124356362925566
Reached leaf Node

Level 4
Count of 0 = 0
Count of 1 = 1
Count of 2 = 0
Current Entropy  is = 0.0
Reached leaf Node

Level 3
Count of 0 = 0
Count of 1 